In [21]:
import os
from ctrace.constraint import load_graph, find_contours
from contact_tracing import PQ
from contact_tracing import *
from ctrace.constraint import ProbMinExposed, ProbMinExposedMIP
from ctrace.draw import *
os.getcwd()

'/home/zach/Documents/prog/research/ContactTracing/ctrace'

In [22]:
LOADING = True
np.random.seed(42)

In [23]:
# Setup graph and infected sets
if LOADING:
    G = load_graph("montgomery")
    # Randomly sample 5% of infected
    N = G.number_of_nodes()
    num_infected = int(N * 0.05)
    I = np.random.choice(N, num_infected, replace=False)

    # Set K value
    K = int(0.8 * len(I))
else:
    G = nx.balanced_tree(3, 3)
    I = {0, 1}

    # Set K value
    K = 3

# Find contours
contour1, contour2 = find_contours(G, I)

# Find the infected probabilities
p1, q = PQ_deterministic(G, I, contour1, 0.1)


In [28]:
from simulation import initial

SIR_init = initial(G=G, timesteps=5, p=0.1, cache="test.json")
print(len(SIR_init[1]))

[  8  14  27  47 120 282]
282


In [29]:
SIR = initial(G, from_cache="test.json")
print(len(SIR["I"]))

282


In [6]:
print(K)
print(p1, q)

3
{2: 0.9, 3: 0.9, 4: 0.9, 5: 0.9, 6: 0.9} defaultdict(<function PQ_deterministic.<locals>.<lambda> at 0x7fefe02e45e0>, {})


In [ ]:
if not LOADING:
    draw_contours(G, I, contour1, contour2)

In [ ]:
print(f"K: {K}")
print(f"I: {I}")

In [ ]:
constraintMIP = ProbMinExposedMIP.from_dataframe(G, I, contour1, contour2, p1, q, K)
constraintNone = ProbMinExposedMIP.from_dataframe(G, I, contour1, contour2, p1, q, 0)

In [ ]:
constraintMIP.solve_lp()
constraintNone.solve_lp()

In [ ]:
constraintMIP.getVariables()

In [ ]:
quarantined = constraintMIP.quarantined_solution
saved = constraintMIP.saved_solution

print(quarantined)
print(saved)

In [ ]:
# Solution Visualization
draw_prob(G, I, contour1, contour2, quarantined, saved, constraintMIP.p1, constraintMIP.q)

In [ ]:
draw_prob(G, I,
          contour1,
          contour2,
          constraintNone.quarantined_solution,
          constraintNone.saved_solution,
          constraintNone.p1,
          constraintNone.q)

In [ ]:
constraintNone.saved_solution

In [ ]:
saved

In [ ]:
[-np.log(1 - x) for _, x in saved.items()]